<a href="https://colab.research.google.com/github/Srikanth9142/ML-works/blob/master/Language%20Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!unzip -q "/content/drive/My Drive/dataset_20.zip"
%ls

bengali_20/  gujarathi_20/  manipuri_20/  sample_data/  urdu_20/
dogri_20/    hindi_20/      marathi_20/   tamil_20/
drive/       malayalam_20/  odisha_20/    telugu_20/


In [0]:
pip install python_speech_features


  Created wheel for python-speech-features: filename=python_speech_features-0.6-cp36-none-any.whl size=5889 sha256=a90e7092b7edd6c5d4cf3ff906f71499781077ee331d1a5452e0b3623de4abe1
  Stored in directory: /root/.cache/pip/wheels/3c/42/7c/f60e9d1b40015cd69b213ad90f7c18a9264cd745b9888134be
Successfully built python-speech-features


In [0]:
from python_speech_features import mfcc,delta,logfbank 

In [0]:
import numpy
import scipy.io.wavfile
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import numpy as np
import wave
import contextlib
from scipy.fftpack import dct
import scipy.io.wavfile as wav

In [0]:
pre_emphasis = 0.97
frame_size = 0.025
frame_stride = 0.01
NFFT = 512
nfilt = 40
num_ceps = 12
cep_lifter =22

In [0]:
def length_audio(folder_name,file_name):
  file_path='/content/'+str(folder_name)+"/"+str(file_name)
  sample_rate, signal = scipy.io.wavfile.read(file_path)
  with contextlib.closing(wave.open(file_path,'r')) as f:
      frames = f.getnframes()
      rate = f.getframerate()
      duration = frames / float(rate)
  return int(duration)

In [0]:
def create_mfcc(folder_name,file_name,start_point):
    file_path ='/content/'+str(folder_name)+"/"+str(file_name)
    sample_rate, signal = scipy.io.wavfile.read(file_path)
    signal = signal[start_point:int(start_point+2 * sample_rate)]   #framing to 2 seconds
    emphasized_signal = numpy.append(signal[0], signal[1:] - pre_emphasis * signal[:-1]) #pre-emphasis
    frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
    signal_length = len(emphasized_signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(numpy.ceil(float(numpy.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame
    pad_signal_length = num_frames * frame_step + frame_length
    z = numpy.zeros((pad_signal_length - signal_length))
    pad_signal = numpy.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal
    indices = numpy.tile(numpy.arange(0, frame_length), (num_frames, 1)) + numpy.tile(numpy.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    frames = pad_signal[indices.astype(numpy.int32, copy=False)]
    #After slicing the signal into frames, we apply a window function such as the Hamming window
    frames *= numpy.hamming(frame_length)  #hamming window
    mag_frames = numpy.absolute(numpy.fft.rfft(frames, NFFT))  # Magnitude of the FFT
    pow_frames = ((1.0 / NFFT) * ((mag_frames) ** 2))  # Power Spectrum
    low_freq_mel = 0
    high_freq_mel = (2595 * numpy.log10(1 + (sample_rate / 2) / 700))  # Convert Hz to Mel
    mel_points = numpy.linspace(low_freq_mel, high_freq_mel, nfilt + 2)  # Equally spaced in Mel scale
    hz_points = (700 * (10**(mel_points / 2595) - 1))  # Convert Mel to Hz
    bin = numpy.floor((NFFT + 1) * hz_points / sample_rate)

    fbank = numpy.zeros((nfilt, int(numpy.floor(NFFT / 2 + 1))))
    for m in range(1, nfilt + 1):
        f_m_minus = int(bin[m - 1])   # left
        f_m = int(bin[m])             # center
        f_m_plus = int(bin[m + 1])    # right

        for k in range(f_m_minus, f_m):
            fbank[m - 1, k] = (k - bin[m - 1]) / (bin[m] - bin[m - 1])
        for k in range(f_m, f_m_plus):
            fbank[m - 1, k] = (bin[m + 1] - k) / (bin[m + 1] - bin[m])

    filter_banks = numpy.dot(pow_frames, fbank.T)
    filter_banks = numpy.where(filter_banks == 0, numpy.finfo(float).eps, filter_banks)  # Numerical Stability
    filter_banks = 20 * numpy.log10(filter_banks)  # dB
    mfcc = dct(filter_banks, type=2, axis=1, norm='ortho')[:, 1 : (num_ceps + 1)] # Keep 2-13
    (nframes, ncoeff) = mfcc.shape
    n = numpy.arange(ncoeff)
    lift = 1 + (cep_lifter / 2) * numpy.sin(numpy.pi * n / cep_lifter)
    mfcc *= lift  #*
    
    filter_banks -= (numpy.mean(filter_banks, axis=0) + 1e-8)
    mfcc -= (numpy.mean(mfcc, axis=0) + 1e-8)
    
    return filter_banks

In [0]:
file_names=['bengali_20','dogri_20','gujarathi_20','hindi_20','malayalam_20','manipuri_20','marathi_20','odisha_20','tamil_20','telugu_20','urdu_20']

In [0]:

def final_data(input_data):
    data_dic = np.zeros((23650,398,40))
    data_tar = np.zeros((23650))
    index = 0
    for inp in input_data:
        mfcc_len=length_audio(file_names[input_data[inp]-1],inp)
        for i in range(0,mfcc_len,2):
            mfcc = create_mfcc(file_names[input_data[inp]-1],inp,i)
            for j in range(len(mfcc)):
              for k in range(len(mfcc[j])):
                data_dic[index][j][k] = mfcc[j][k]
            data_tar[index] = input_data[inp]
            index += 1
            # data_dict.append(mfcc)
            # data_tar.append(input_data[inp]) 
    return data_dic,data_tar

In [0]:

languageName =["bengali","dogri","gujarathi","hindi","malayalam","manipuri","marathi","odisha","tamil","telugu","urdu"]
languageEntries = [253,181,178,179,125,154,364,300,242,183,206]
languageCode = [1,2,3,4,5,6,7,8,9,10,11]
input_data = {}
for i in range(len(languageName)):
    for j in range(1,languageEntries[i]+1):
        input_data[languageName[i]+str(j)+".wav"] = languageCode[i]

In [0]:
##################################################################################################
input_data_output_dict,input_data_output_tar=final_data(input_data)
##################################################################################################

In [0]:
tarModified = np.zeros((23650,11))
iter = 0
for i in input_data_output_tar:
  index = int(i)
  tarModified[iter][index-1] = 1
  iter += 1

In [0]:
print(len(input_data_output_dict))
print(len(tarModified))

NameError: ignored

In [0]:
# Import train_test_split function
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, Y_train, Y_test = train_test_split(input_data_output_dict,tarModified, test_size=0.25,random_state=109) # 75% training and 30% test

In [0]:
X_train=np.asarray(X_train)
Y_train=np.asarray(Y_train)
X_test=np.asarray(X_test)
Y_test=np.asarray(Y_test)
print("X_train : "+str(X_train.shape)+str(X_train[0].shape))
print("X_test : "+str(X_test.shape)+str(X_test[0].shape))
print("Y_train : "+str(Y_train.shape))
print("Y_test : "+str(Y_test.shape))

X_train : (17737, 398, 40)(398, 40)
X_test : (5913, 398, 40)(398, 40)
Y_train : (17737, 11)
Y_test : (5913, 11)


In [0]:
from keras import applications
base_model=applications.resnet50.ResNet50(weights=None,include_top=False,input_shape = (40,40,1))
base_model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 40, 40, 1)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 46, 46, 1)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 20, 20, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 20, 20, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
from keras.layers import GlobalAveragePooling2D,Dropout,Dense,BatchNormalization,Activation,AveragePooling2D,Flatten
from keras.models import Model
x = base_model.output

In [0]:
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = AveragePooling2D(pool_size=2)(x)
y = Flatten()(x)
outputs = Dense(11,activation='softmax',kernel_initializer='he_normal')(y)
rnet=Model(inputs=base_model.input,outputs=outputs)
rnet.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 40, 40, 1)    0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 46, 46, 1)    0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 20, 20, 64)   3200        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 20, 20, 64)   256         conv1[0][0]                      
____________________________________________________________________________________________

In [0]:
from keras import applications
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Dense, Dropout, Activation, Flatten, Add, BatchNormalization,AveragePooling2D
from keras.layers import Convolution2D, MaxPooling2D
from keras.layers import GlobalAveragePooling2D,Dropout,Dense
from keras.models import Model

In [0]:
model = Sequential()
model.add(Convolution2D(1,(3,3),padding='same',activation = 'relu',input_shape = (398,40,1)))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(MaxPooling2D(pool_size=(2, 1)))
model.add(MaxPooling2D(pool_size=(2, 1)))

#model.add(Convolution2D(1, (3,3),strides=(2,1), padding='same',activation='relu',use_bias=False))

#model.add(Convolution2D(1, 3, 3, activation='relu',use_bias=False))
model.add(Convolution2D(1, 2, 1, activation='relu',use_bias=False))
model.add(Convolution2D(1, 3, 1, activation='relu',use_bias=False))
model.add(Convolution2D(1, 3, 1, activation='relu',use_bias=False))
model.add(Convolution2D(1, 3, 1, activation='relu',use_bias=False))
model.add(Convolution2D(1, 3, 1, activation='relu',use_bias=False))
#model.add(Convolution2D(32, 3, 3, activation='relu',use_bias=False))

#model.add(Flatten())
#model.add(Activation('softmax'))
model.add(rnet)
model.summary()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (2, 1), activation="relu", use_bias=False)`
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (3, 1), activation="relu", use_bias=False)`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (3, 1), activation="relu", use_bias=False)`
  if sys.path[0] == '':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1, (3, 1), activation="relu", use_bias=False)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 398, 40, 1)        10        
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 199, 40, 1)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 99, 40, 1)         0         
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 49, 40, 1)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 40, 1)         2         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 46, 40, 1)         3         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 44, 40, 1)        

In [0]:
from keras.optimizers import Adam
adam=Adam(lr=0.0001)
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [0]:

x_train_reshaped = X_train.reshape(X_train.shape[0],398,40,1)

In [0]:
model.fit(x_train_reshaped,Y_train,epochs=25,batch_size=64)

NameError: ignored

In [0]:
from keras.models import load_model
model.save('language_classificationII.h5')

Using TensorFlow backend.


NameError: ignored

In [0]:
x_test = X_test.reshape(X_test.shape[0],398,40,1)
y_pred=model.predict(x_test)

In [0]:
import keras
# from sklearn.metrics import accuracy_score
# print(accuracy_score(Y_test, y_pred))
acc=0
for i in range(len(Y_test)):
  count=0
  for j in range(len(Y_test[i])):
    if(Y_test[i][j]==round(y_pred[i][j],0)):
      count+=1
  if(count==11):
    acc+=1
print("Accuracy is",(acc/len(Y_test))*100)

NameError: ignored

In [0]:
def prediction(targ):
    accents = ['bengali','dogri','gujarathi','hindi','malayalam','manipuri','marathi','odisha','tamil','telugu','urdu']
    np.set_printoptions(formatter={'float_kind':'{:.1f}'.format})

In [0]:

def Pre_final_data(folder,files):
    index=0
    data_dic = np.zeros((23650,398,40))
    mfcc=[]
    target=[]
    mfcc_len=length_audio(folder,files)
    
    for i in range(0,mfcc_len,2):
        
        mfcc = create_mfcc(folder,files,i)
        for j in range(len(mfcc)):
              for k in range(len(mfcc[j])):
                data_dic[index][j][k] = mfcc[j][k]
        index += 1
        #target.append(mfcc)
    #target=np.asarray()
    
    data_dic_reshaped = data_dic.reshape(data_dic.shape[0],398,40,1)
    targ=model.predict(data_dic_reshaped)    
    print(targ)
    prediction(targ)

In [0]:
folder='drive/My Drive/testing'
file='test_sri.wav'
print(Pre_final_data(folder,file))

NameError: ignored